In [1]:
#!/usr/bin/env python3
# Copyright 2022 ETH Zurich and University of Bologna.
# Licensed under the Apache License, Version 2.0, see LICENSE for details.
# SPDX-License-Identifier: Apache-2.0

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import argparse
import pathlib
import hjson
import random
import os

In [2]:
np.random.seed(42)
torch.manual_seed(42)
global verbose

In [3]:
def array_to_cstr(a):
    out = '{'
    if isinstance(a, np.ndarray):
        a = a.flat
    if isinstance(a, torch.Tensor):
        a = a.numpy().flat
    for el in a:
        out += '{}, '.format(el)
    out = out[:-2] + '}'
    return out

In [4]:
def emit_mnist_data(name='mnist', **kwargs):
    
    # constants
    IN_CH = kwargs['IN_CH']
    OUT_CH = kwargs['OUT_CH']
    DATASET_SIZE = kwargs['DATASET_SIZE']
    
    # data
    MAT_INPUT = kwargs['INPUT']
    MAT_LABELS = kwargs['LABELS']

    # network init parameters from golden model
    MAT_WEIGHTS = kwargs['WEIGHTS']
    MAT_BIASES = kwargs['BIASES'] 
    
    layer_str = ''
    layer_str += '#include "network.h"\n\n'
    layer_str += f'network_benchmark_t {name}_t = {{\n'
    layer_str += f'\t.IN_CH = {IN_CH},\n'
    layer_str += f'\t.OUT_CH = {OUT_CH},\n'
    layer_str += f'\t.dtype = FP{kwargs["prec"]}\n'
    layer_str += '};\n\n\n'

    ctypes = {
        '64': 'double',
        '32': 'float',
        '16': '__fp16',
        'B16': '__bf16',
        '8': 'char'
    }

    dtype = ctypes[str(kwargs['prec'])]

    # network initialization
    layer_str += f'static {dtype} {name}_weights_dram [{OUT_CH}][{IN_CH}] = ' + array_to_cstr(MAT_WEIGHTS) + ';\n\n\n'
    layer_str += f'static {dtype} {name}_biases_dram [{OUT_CH}][{1}] = ' + array_to_cstr(MAT_BIASES) + ';\n\n\n'


    # input data
    layer_str += f'static {dtype} {name}_images_dram [{DATASET_SIZE*IN_CH}][{1}] = ' + array_to_cstr(MAT_INPUT) + ';\n\n\n'
    layer_str += f'static uint32_t {name}_labels_dram [{DATASET_SIZE}][{1}] = ' + array_to_cstr(MAT_LABELS) + ';\n\n\n'

    return layer_str


In [6]:
def emit_mnist_header_file(layer_type: str, **kwargs):

    file_path = '/scratch/msc22f11/msc22f11/snitch/sw/applications/data/'
    emit_str = "// Copyright 2022 ETH Zurich and University of Bologna.\n" + \
               "// Licensed under the Apache License, Version 2.0, see LICENSE for details.\n" + \
               "// SPDX-License-Identifier: Apache-2.0\n\n"

    if(layer_type == 'mnist'):
        file = file_path + 'data_fp32_benchmark.h'
        emit_str += emit_mnist_data(**kwargs)

    with open(file, 'w') as f:
        f.write(emit_str)


In [7]:
def Linear(input, weights, bias, **kwargs):
    out = torch.mul(input, weights)
    return out

In [8]:
# download MNIST dataset using DataLoader

transform = transforms.Compose(
    [
        transforms.ToTensor()
    ]
)

PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
mnist_dataset = MNIST(PATH_DATASETS, train=True, transform=transform, download=True)

# set seeds for reproducability 
g = torch.Generator()
g.manual_seed(42)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

mnist_dl = DataLoader(mnist_dataset, worker_init_fn=seed_worker, generator=g)

In [9]:
first_im, first_label = next(iter(mnist_dl))

In [10]:
np.random.seed(42)
torch.manual_seed(42)
# get input channels
IN_CH = 1 * 28 * 28 # 3 channels, 32x32 pixels
OUT_CH = 16 # 16 classes
r1 = 0
r2 = 0.5

# get random input data with shape (IN_CH, 1)
input = first_im.to(torch.float32).view(first_im.to(torch.float64).size(0), -1) #torch.randn(IN_CH)
print(input.shape)

# get random weights with shape (OUT_CH, IN_CH)
weights = torch.FloatTensor(OUT_CH, IN_CH).uniform_(r1, r2).to(torch.float32) #torch.randn(OUT_CH, IN_CH).to(torch.float64)
print(weights.shape)

# get random bias with shape (OUT_CH, 1)
bias = torch.FloatTensor(OUT_CH).uniform_(r1, r2).to(torch.float32)#torch.randn(OUT_CH).to(torch.float64)
print(bias.shape)

# calculate the activations of the linear layer
activations = input @ weights.t() + bias
print(activations.shape)
# get a random integer between 0 and 16
label = torch.randint(0, 16, (1,))
print(label)

torch.Size([1, 784])
torch.Size([16, 784])
torch.Size([16])
torch.Size([1, 16])
tensor([7])


In [12]:
print(activations)
# print data type of the activations
print(activations.dtype)

tensor([[26.3321, 26.9016, 27.8120, 30.3491, 27.2808, 25.2449, 28.9580, 26.0793,
         28.7988, 27.2119, 25.8306, 28.1869, 27.7081, 24.9478, 27.7745, 29.6327]])
torch.float32


In [13]:
np.max(activations.numpy())

30.349098

In [14]:
# apply softmax to the activations
softmax = torch.nn.Softmax(dim=1)
ff_out = softmax(activations)
print(ff_out)
print(ff_out.shape)
print(ff_out.dtype)

tensor([[0.0073, 0.0129, 0.0321, 0.4055, 0.0189, 0.0025, 0.1009, 0.0057, 0.0860,
         0.0176, 0.0044, 0.0467, 0.0289, 0.0018, 0.0309, 0.1981]])
torch.Size([1, 16])
torch.float32


In [15]:
# transform softmax activations to list
ff_out_l = ff_out.tolist()[0]
# if index matches label, subtract 1 from value at index
ff_out_l[label] = ff_out_l[label] - 1
# print the bias gradient 
bias_gradients = torch.FloatTensor(ff_out_l).reshape(1, -1)
print(bias_gradients)
print(bias_gradients.shape)
print(bias_gradients.dtype)

tensor([[ 0.0073,  0.0129,  0.0321,  0.4055,  0.0189,  0.0025,  0.1009, -0.9943,
          0.0860,  0.0176,  0.0044,  0.0467,  0.0289,  0.0018,  0.0309,  0.1981]])
torch.Size([1, 16])
torch.float32


In [16]:
# compute the weight gradient matrix
weight_gradients = torch.mul(input.t(), bias_gradients).t()
# compute the checksum for every column of the weight gradient matrix
weight_gradients_checksum = torch.sum(weight_gradients, dim=1)
print(weight_gradients_checksum)
print(weight_gradients_checksum.shape)
print(weight_gradients_checksum.dtype)

tensor([   0.7881,    1.3928,    3.4617,   43.7660,    2.0351,    0.2657,
          10.8887, -107.3291,    9.2861,    1.8996,    0.4773,    5.0363,
           3.1202,    0.1974,    3.3343,   21.3798])
torch.Size([16])
torch.float32


In [22]:
# compute the training step
bias_update = bias - torch.mul(bias_gradients, 0.5)
print("bias_update = ", bias_update)
print(bias_update.shape)
print(bias_update.dtype)
weight_update = weights - torch.mul(weight_gradients, 0.5)
weight_update_checksum = torch.sum(weight_update, dim=1)
print("\nweight_update_checksum = ", weight_update_checksum)
print(weight_update_checksum.shape)
print(weight_update_checksum.dtype)

bias_update =  tensor([[ 0.3965,  0.4461,  0.3087,  0.2464,  0.1215,  0.0852, -0.0012,  0.7076,
          0.1823,  0.0991,  0.1623,  0.1462,  0.4767,  0.2312,  0.4372,  0.1750]])
torch.Size([1, 16])
torch.float32

weight_update_checksum =  tensor([188.4179, 197.7575, 191.1750, 176.6682, 190.4993, 195.2947, 197.8444,
        248.8347, 197.1029, 191.0635, 198.1736, 196.9793, 192.9529, 189.7249,
        194.1348, 181.0583])
torch.Size([16])
torch.float32


In [19]:
# calculate the memory requirements
if(activations.dtype == torch.float64):
    print(f'Input size: {IN_CH * 64 / 8 / 1024} KB')
    print(f'Weights size: {OUT_CH * IN_CH * 64 / 8 / 1024} KB')
    print(f'Bias size: {OUT_CH * 64 / 8 / 1024} KB')
    print(f'Output size: {OUT_CH * 64 / 8 / 1024} KB')
    print(f'\nTotal size: {(IN_CH + OUT_CH * IN_CH + OUT_CH) * 64 / 8 / 1024} KB')
elif(activations.dtype == torch.float32):
    print(f'Input size: {IN_CH * 32 / 8 / 1024} KB')
    print(f'Weights size: {OUT_CH * IN_CH * 32 / 8 / 1024} KB')
    print(f'Bias size: {OUT_CH * 32 / 8 / 1024} KB')
    print(f'Output size: {OUT_CH * 32 / 8 / 1024} KB')
    print(f'\nTotal size: {(IN_CH + OUT_CH * IN_CH + OUT_CH) * 32 / 8 / 1024} KB')

Input size: 3.0625 KB
Weights size: 49.0 KB
Bias size: 0.0625 KB
Output size: 0.0625 KB

Total size: 52.125 KB


In [20]:
kwargs = {
            'IN_CH': IN_CH,
            'OUT_CH': OUT_CH,
            'DATASET_SIZE': 1,
            'INPUT': input,
            'WEIGHTS': weights.detach(),
            'BIASES': bias.detach(),
            'LABELS': label,
            'prec': 32
}

In [21]:
emit_mnist_header_file('mnist', **kwargs)